In [1]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:

import numpy as np
import pandas as pd

In [4]:
data_path = os.path.join(module_path, 'Data', 'artigos.csv')
df = pd.read_csv(data_path)

df = pd.read_csv(data_path, delimiter=',')
df.columns = [col.strip().replace('"', '') for col in df.columns]

df.head()


,ID,Title,Abstract,Keywords,Authors,Year,ISD,DOI,Links,Publication Type,Search Engine,VALIDAÇÃO,PDF
0,1535,Reconstructing Missing EHRs Using Time-Aware W...,Real-world Electronic Health Records (EHRs) ar...,"Electronic Health Records(EHRs), EHRs Imputati...",G. Gao; F. Khoshnevisan; M. Chi,2022,2575-2634,10.1109/ICHI54592.2022.00034,https://ieeexplore.ieee.org/stamp/stamp.jsp?ar...,IEEE Conferences,IEEE,AC,2203.08245v2.pdf
1,16,3D-MICE: integration of cross-sectional and lo...,A key challenge in clinical data mining is tha...,"machine learning, imputation, missing data, el...",Baron JM,2018,1527-974X,10.1093/jamia/ocx133,https://pubmed.ncbi.nlm.nih.gov/29202205/,"Journal Article Research Support, Non-U.S. Gov't",PubMed,AC,3D-MICE.pdf
2,1385,Performance evaluation of a recurrent deep neu...,Atmospheric pollution refers to the presence o...,NaN,Pedraza-Ortega JC,2022,2162-2906,10.1080/10962247.2022.2095057,https://pubmed.ncbi.nlm.nih.gov/35816429/,Journal Article,PubMed,AC,Performance evaluation of a recurrent deep neu...
3,1433,Predicting progression of Alzheimer's disease ...,"If left untreated, Alzheimer's disease (AD) is...","Alzheimer’s progression, MRI biomarker forecas...",Pant S,2022,1879-2782,S0893-6080(22)00094-6,https://pubmed.ncbi.nlm.nih.gov/35364417/,Journal Article,PubMed,AC,1-s2.0-S0893608022000946-main.pdf
4,369,Causes and Consequences of Missing Health-Rela...,Missing health-related quality of life (HRQOL)...,"longitudinal studies , quality of life , regis...",Spertus JA,2017,1941-7705,10.1161/CIRCOUTCOMES.116.003268,https://pubmed.ncbi.nlm.nih.gov/29246883/,Comparative Study Journal Article Multicenter ...,PubMed,AC,grady2017.pdf


In [5]:
abstracts = df[['ID', 'Abstract']].values


In [6]:
abstracts[0]

array([1535,
       'Real-world Electronic Health Records (EHRs) are often plagued by a high rate of missing data. In our EHRs, for example, the missing rates can be as high as 90% for some features, with an average missing rate of around 70% across all features. We propose a Time-Aware Dual-Cross-Visit missing value imputation method, named TA-DualCV, which spontaneously leverages multivariate dependencies across features and longitudinal dependencies both within- and cross-visit to maximize the information extracted from limited observable records in EHRs. Specifically, TA-DualCV captures the latent structure of missing patterns across measurements of different features and it also considers the time continuity and capture the latent temporal missing patterns based on both time-steps and irregular time-intervals. TA-DualCV is evaluated using three large real-world EHRs on two types of tasks: an unsupervised imputation task by varying mask rates up to 90% and a supervised 24-hour earl

In [ ]:
import nltk
import string
from nltk.corpus import stopwords
import re

# Baixar stopwords em português
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Palavras adicionais a serem removidas
custom_stopwords = [
    "time series", "temporal", "reconstruct", 
    "missing values", "missing data", "incomplete data"
]

# Regex para capturar palavras que começam com "multi" (case insensitive)
multi_regex = re.compile(r'\bmulti\w*', flags=re.IGNORECASE)
imput_regex = re.compile(r'\bimput\w*', flags=re.IGNORECASE)

def preprocess_text(text):
    # Converter para minúsculas
    text = text.lower()
    # Remover pontuação
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remover palavras compostas específicas
    for phrase in custom_stopwords:
        text = text.replace(phrase, "")
    # Remover palavras que começam com "multi"
    text = re.sub(multi_regex, '', text)
    text = re.sub(imput_regex, '', text)
    # Remover stopwords simples
    tokens = [word for word in text.split() if word not in stop_words]
    return ' '.join(tokens)

# Aplicar ao DataFrame (considerando que temos a coluna 'Abstract')
df['cleaned_abstract'] = df['Abstract'].apply(preprocess_text)

# Ver abstracts limpos
print(df['cleaned_abstract'].head())


0    realworld electronic health records ehrs often...
1    key challenge clinical data mining clinical da...
2    atmospheric pollution refers presence substanc...
3    left untreated alzheimers disease ad leading c...
4    missing healthrelated quality life hrqol data ...
Name: cleaned_abstract, dtype: object


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igorc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [9]:
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform(df['cleaned_abstract'])  # Document-Term Matrix


print(vectorizer.get_feature_names_out())

['0007' '001' '00130015' ... 'york' 'zealand' 'zone']


In [10]:
from sklearn.decomposition import LatentDirichletAllocation


K = 5 


lda_model = LatentDirichletAllocation(n_components=K, random_state=42)
lda_model.fit(dtm)


for index, topic in enumerate(lda_model.components_):
    print(f"Tópico {index + 1}:")
    print([vectorizer.get_feature_names_out()[i] for i in topic.argsort()[-10:]])

Tópico 1:
['mean', 'patients', 'prediction', 'septic', 'longitudinal', 'scores', 'shock', 'data', 'hrqol', 'missing']
Tópico 2:
['analysis', 'dependencies', 'propose', 'model', 'time', 'graph', 'ta', 'method', 'missing', 'data']
Tópico 3:
['professionalization', 'mice', 'missing', '3dmice', 'nurse', 'data', 'gender', 'results', 'nursing', 'clinical']
Tópico 4:
['knowledge', 'one', 'however', 'often', 'learning', 'challenge', 'architecture', 'model', 'clinical', 'data']
Tópico 5:
['neural', 'results', 'aod', 'methodology', 'model', 'deep', 'modeling', 'using', 'network', 'data']


In [11]:
# Distribuição de tópicos por documento
doc_topic_dist = lda_model.transform(dtm)


topic_df = pd.DataFrame(doc_topic_dist, columns=[f"Tópico {i+1}" for i in range(K)])
topic_df['Documento'] = df['ID']  # Adicionar identificação do documento






topic_df


,Tópico 1,Tópico 2,Tópico 3,Tópico 4,Tópico 5,Documento
0,0.001831,0.992729,0.001812,0.001811,0.001817,1535
1,0.001303,0.001304,0.994786,0.001306,0.001301,16
2,0.000894,0.000895,0.000894,0.000894,0.996423,1385
3,0.994646,0.001347,0.001332,0.001335,0.001339,1433
4,0.995163,0.001210,0.001210,0.001207,0.001211,369
5,0.002121,0.991514,0.002115,0.002121,0.002129,375
6,0.001414,0.166300,0.001409,0.001419,0.829457,447
7,0.993225,0.001693,0.001693,0.001694,0.001696,1656
8,0.001297,0.001297,0.001303,0.994805,0.001297,718
9,0.002642,0.989413,0.002642,0.002652,0.002651,1282


In [12]:
topic_df.to_csv('topicos.csv', index=False)

In [13]:
import pyLDAvis
import pyLDAvis.lda_model


# Visualizar os tópicos
pyLDAvis.enable_notebook()
lda_vis = pyLDAvis.lda_model.prepare(lda_model, dtm, vectorizer)
pyLDAvis.display(lda_vis)

c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\joblib\_utils.py:39: DeprecationWarning: Attribute n is deprecated and will be removed in Python 3.14; use value instead
  return node.n
c:\Python312\Lib\site-packages\joblib\_utils.py:38: DeprecationWarning: ast.Num is deprecated and will be removed in Python 3.14; use ast.Constant instead
  if isinstance(node, ast.Num):  # <number>
c:\Python312\Lib\site-packages\jobli

In [ ]:
pyLDAvis.save_html(lda_vis, 'lda_visualization.html')

: 